In [ ]:
import os
import json
import pretty_midi
from midi_player import MIDIPlayer
from midi_player.stylers import basic
from corpus.tokenizer_v2 import MidiTokenizer

def json_to_midi(notes, output_path):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)

    for note in notes:
        instrument.notes.append(
            pretty_midi.Note(
                velocity=note["velocity"],
                pitch=note["pitch"],
                start=note["onset"],
                end=note["offset"],
            )
        )
    midi.instruments.append(instrument)
    midi.write(output_path)

def json_to_midi_with_beat(notes, output_path, beat_json_path):
    midi = pretty_midi.PrettyMIDI()
    
    instrument = pretty_midi.Instrument(program=0)
    for note in notes:
        instrument.notes.append(
            pretty_midi.Note(
                velocity=note["velocity"],
                pitch=note["pitch"],
                start=note["onset"],
                end=note["offset"],
            )
        )
    midi.instruments.append(instrument)
    
    if not os.path.exists(beat_json_path):
        print(f"Missing beat info JSON file: {beat_json_path}")
        return
    with open(beat_json_path, "r") as f:
        beat_info = json.load(f)
    
    metronome_track = pretty_midi.Instrument(
        program=115, is_drum=True, name="Metronome"
    )
    beat_duration = 0.1 
    for downbeat in beat_info.get("downbeat_pred", []):
        note_downbeat = pretty_midi.Note(
            velocity=100,
            pitch=36,
            start=downbeat,
            end=downbeat + beat_duration,
        )
        metronome_track.notes.append(note_downbeat)
    
    for beat in beat_info.get("beat_pred", []):
        note_beat = pretty_midi.Note(
            velocity=100,
            pitch=38,
            start=beat,
            end=beat + beat_duration,
        )
        metronome_track.notes.append(note_beat)
    
    midi.instruments.append(metronome_track)
    
    midi.write(output_path)


def get_midi_player(midi_file_path):
    return MIDIPlayer(url_or_file=midi_file_path, height=600, styler=basic, title='My Player')

base_folder = "../../dataset/0000/1/"
midi_path = base_folder + "aligned_transcription.json"
tempo_path = base_folder + "tempo.json"
beats_path = base_folder + "beats.json"
midi_file_path = base_folder + "detokenize.mid"

tokenizer = MidiTokenizer(tempo_path)
events = tokenizer.encode(midi_path, with_grace_note=True)
for e in events:
    print(e)

# tokenizer.decode_to_score(events)
# restored_notes = tokenizer.restore()

tokenizer = MidiTokenizer(tempo_path)
decoded_notes = tokenizer.decode_to_notes(events)

json_to_midi(decoded_notes, midi_file_path)
get_midi_player(midi_file_path)

# with open(midi_path, 'r') as f:
#     notes = json.load(f)

# json_to_midi_with_beat(decoded_notes, midi_file_path, beats_path)
# get_midi_player(midi_file_path)

In [ ]:
from midi_player import MIDIPlayer
from midi_player.stylers import basic

def get_midi_player(midi_file_path):
    return MIDIPlayer(url_or_file=midi_file_path, height=600, styler=basic, title='My Player')

out_dir = "./outputs/inference/"
eval_dir = "./dataset/eval/"
src_dir = "./infer/src/"

extract_path = src_dir + "temp/extract.json"
midi_file_path = out_dir + "etude_d.mid"

# with open(extract_path, "r") as f:
#     notes = json.load(f)
# with open(origin_path, "r") as f:
#     notes = json.load(f)

get_midi_player(midi_file_path)

In [ ]:
from midi_player import MIDIPlayer
from midi_player.stylers import basic

def get_midi_player(midi_file_path):
    return MIDIPlayer(url_or_file=midi_file_path, height=600, styler=basic, title='My Player')

out_dir = "./infer/output/"
eval_dir = "./dataset/eval/"
src_dir = "./infer/src/"

extract_path = src_dir + "extract.json"
midi_file_path = "../PiCoGen-2/data/processed/0003/piano.mid"

# with open(extract_path, "r") as f:
#     notes = json.load(f)
# with open(origin_path, "r") as f:
#     notes = json.load(f)

get_midi_player(midi_file_path)

In [ ]:
#!/usr/bin/env python3
# filename: trim_wav.py

import argparse
from pydub import AudioSegment

def trim_wav(input_path: str, output_path: str, trim_seconds: float):
    # 讀取 WAV
    audio = AudioSegment.from_wav(input_path)
    # 裁掉前面 trim_seconds 秒（pydub 單位為毫秒）
    trimmed = audio[int(trim_seconds * 1000):]
    # 輸出新的 WAV
    trimmed.export(output_path, format="wav")
    print(f"✅ 完成裁切：從 {trim_seconds} 秒開始輸出至 {output_path}")

def trim_wav_tail(input_path: str, output_path: str, tail_seconds: float):
    """
    裁切 WAV 檔案尾端的 tail_seconds 秒，
    只保留從開頭到 (總長度 - tail_seconds) 的音訊。
    """
    # 讀取 WAV
    audio = AudioSegment.from_wav(input_path)
    duration_ms = len(audio)  # 總長度（毫秒）
    trim_ms = int(tail_seconds * 1000)

    if trim_ms >= duration_ms:
        raise ValueError(f"裁切秒數 {tail_seconds}s 超過或等於檔案總長度 {duration_ms/1000:.2f}s")

    # 只保留前面的 (duration_ms - trim_ms) 毫秒
    trimmed = audio[: duration_ms - trim_ms]
    # 輸出新的 WAV
    trimmed.export(output_path, format="wav")
    print(f"✅ 已裁切尾端 {tail_seconds} 秒，輸出到：{output_path}")

target = "./dataset/eval/CPOP17/cover.wav"
# trim_wav(target, target, 10)
# trim_wav_tail(target, target, 20)


In [ ]:
from pydub import AudioSegment

def trim_one_minute_from(input_path: str, output_path: str, start_time_sec: float):
    """
    從輸入音訊檔案中，裁切從 `start_time_sec` 開始的 60 秒音訊，輸出為新檔案。
    
    參數：
    - input_path: 音訊輸入檔案路徑
    - output_path: 輸出檔案路徑
    - start_time_sec: 要裁切的開始時間（秒）
    """
    audio = AudioSegment.from_wav(input_path)
    start_ms = int(start_time_sec * 1000)
    end_ms = start_ms + 60_000

    if start_ms >= len(audio):
        raise ValueError(f"開始時間 {start_time_sec}s 已超過音訊長度 {len(audio)/1000:.2f}s")
    
    # 若超出範圍，只裁到音檔結尾
    trimmed = audio[start_ms:min(end_ms, len(audio))]

    trimmed.export(output_path, format="wav")
    print(f"✅ 已從 {start_time_sec}s 開始裁切 1 分鐘，輸出至 {output_path}")

start_time = 0
target = "3"
trim_one_minute_from(
    f"./user/Western/{target}/etude_d_d.wav",
    f"./user/Western/{target}/etude_d_d.wav",
    start_time_sec=start_time
)
trim_one_minute_from(
    f"./user/Western/{target}/origin.wav",
    f"./user/Western/{target}/origin.wav",
    start_time_sec=start_time
)
trim_one_minute_from(
    f"./user/Western/{target}/picogen.wav",
    f"./user/Western/{target}/picogen.wav",
    start_time_sec=0
)
trim_one_minute_from(
    f"./user/Western/{target}/picogen-e.wav",
    f"./user/Western/{target}/picogen-e.wav",
    start_time_sec=0
)

In [ ]:
from utils.midi_tool import midi_to_wav, align_and_play_stereo

dir_name = "WESTERN22"
name = "etude_e"
name = "etude_d"

input_midi_path = f"./infer/output/{name}.mid"
output_wav_path = f"./infer/output/{name}.wav"
origin_wav_path = f"./dataset/eval/{dir_name}/origin.wav"
sound_font_path = "./utils/sound_font/SGM-v2.01.sf2"

midi_to_wav(input_midi_path, output_wav_path, sound_font_path)

align_and_play_stereo(origin_wav_path, output_wav_path, "output.wav")

In [ ]:
import json
import os
import numpy as np
import librosa
import soundfile as sf

# 假設您的 MIDI 轉換工具函式位於此處
from utils.midi_tool import midi_to_wav

def normalize_and_overwrite(audio_path, target_peak=0.98):
    """
    載入指定的 WAV 檔案，將其音量正規化後，覆蓋原始檔案。

    Args:
        audio_path (str): 要處理的音訊路徑。
        target_peak (float): 目標峰值振幅，設為略小於 1.0 可避免削波。
    """
    try:
        Fs = 44100 # 確保取樣率與您的專案一致
        audio, _ = librosa.load(audio_path, sr=Fs)
        
        # 如果音訊振幅已經很正常或幾乎是靜音，則跳過以節省時間
        max_amplitude = np.max(np.abs(audio))
        if max_amplitude > 0.9 or max_amplitude < 1e-6:
            # print(f"  - 音量已在正常範圍，無需正規化: {os.path.basename(audio_path)}")
            return

        # 計算正規化因子並應用
        normalized_audio = audio / max_amplitude * target_peak
        
        # 使用 soundfile 覆蓋原始檔案
        sf.write(audio_path, normalized_audio, Fs)
        # print(f"  - 音量已成功正規化: {os.path.basename(audio_path)}")

    except Exception as e:
        print(f"  ❌ 正規化失敗: {audio_path}，錯誤: {e}")


def convert_and_normalize_all_versions():
    """
    讀取 metadata.json，為指定的 MIDI 版本批次轉換為 WAV 檔案，
    並立即對其進行音量正規化。如果目標 WAV 檔案已存在，則跳過。
    """
    # --- 1. 設定路徑和常數 ---
    base_dir = os.path.join(".", "dataset", "eval")
    metadata_path = os.path.join(base_dir, "metadata.json")
    sound_font_path = os.path.join(".", "utils", "sound_font", "SGM-v2.01.sf2")
    
    VERSION_MAPPING = {
        "etude_e": "etude_e",
        "etude_d": "etude_d",
        "etude_d_d": "etude_d_d",
        "picogen": "picogen",
        "amtapc": "amtapc",
        "music2midi": "music2midi",
        "cover": "human"
    }

    # --- 2. 讀取 metadata.json ---
    try:
        with open(metadata_path, 'r', encoding='utf-8') as f:
            metadata = json.load(f)
        print(f"✅ 成功讀取 metadata.json，共找到 {len(metadata)} 首歌曲。")
    except Exception as e:
        print(f"❌ 讀取 metadata.json 失敗: {e}")
        return

    # --- 3. 遍歷歌曲和版本，進行轉換與正規化 ---
    total_processed = 0
    for i, song_data in enumerate(metadata):
        dir_name = song_data.get("dir_name")
        if not dir_name:
            print(f"⚠️ 警告：第 {i+1} 筆資料缺少 'dir_name'，已跳過。")
            continue

        print(f"\n🎵 正在處理歌曲目錄: {dir_name} ({i+1}/{len(metadata)})")

        for input_name, output_name in VERSION_MAPPING.items():
            input_midi_path = os.path.join(base_dir, dir_name, f"{input_name}.mid")
            output_wav_path = os.path.join(base_dir, dir_name, f"{output_name}.wav")

            # --- 【關鍵修改】重新加入檢查，若目標 WAV 檔案已存在，則跳過 ---
            if os.path.exists(output_wav_path):
                print(f"  ↪️ 已跳過 (WAV 已存在): {os.path.basename(output_wav_path)}")
                continue
            
            # 檢查來源 MIDI 是否存在
            if not os.path.exists(input_midi_path):
                # 因為上面已經跳過已存在的檔案，這裡的訊息可以選擇性關閉，避免過多輸出
                # print(f"  ↪️ 已跳過 (MIDI 不存在): {input_midi_path}")
                continue
            
            try:
                # 步驟 A: 執行 MIDI -> WAV 轉換
                print(f"  🔄 正在轉換: {os.path.basename(input_midi_path)} -> {os.path.basename(output_wav_path)}")
                midi_to_wav(input_midi_path, output_wav_path, sound_font_path)
                
                # 步驟 B: 立刻對剛生成的 WAV 檔案進行音量正規化並覆蓋
                normalize_and_overwrite(output_wav_path)
                
                print(f"  ✅ 成功轉換並正規化: {os.path.basename(output_wav_path)}")
                total_processed += 1
            except Exception as e:
                print(f"  ❌ 處理失敗: {input_midi_path}，錯誤: {e}")

    print(f"\n🎉 --- 所有任務已完成！總共處理了 {total_processed} 個新檔案。 ---")


# --- 安裝必要的函式庫 ---
# 如果您尚未安裝 librosa 或 soundfile，請執行:
# pip install librosa soundfile

# --- 執行主程式 ---
if __name__ == '__main__':
    convert_and_normalize_all_versions()

In [ ]:
import os
import sys
import numpy as np
import librosa
import soundfile as sf

# 確保您已安裝必要的函式庫:
# pip install librosa soundfile numpy

# 假設您的 MIDI 轉換工具函式位於此處
# 您需要確保 utils/midi_tool.py 檔案和 midi_to_wav 函式是可用的。
try:
    from utils.midi_tool import midi_to_wav
except ImportError:
    print("❌ 錯誤：無法從 'utils.midi_tool' 導入 'midi_to_wav' 函式。")
    print("請確保您的專案中有 'utils/midi_tool.py' 檔案，並且其中包含 'midi_to_wav' 函式。")
    sys.exit(1)


def normalize_wav_file(audio_path, target_peak=0.98):
    """
    載入指定的 WAV 檔案，將其音量正規化後，覆蓋原始檔案。

    Args:
        audio_path (str): 要處理的音訊路徑。
        target_peak (float): 目標峰值振幅，設為略小於 1.0 可避免削波。
    """
    try:
        # 使用 librosa 載入音訊，sr=None 會保留原始取樣率
        audio, sr = librosa.load(audio_path, sr=None)
        
        # 找到音訊的最大振幅
        max_amplitude = np.max(np.abs(audio))
        
        # 如果音訊是靜音或振幅已經很大，則跳過以節省時間並避免錯誤
        if max_amplitude < 1e-6 or max_amplitude > 0.95:
            print(f"    - 音量無需正規化 (靜音或已足夠大)。")
            return

        # 計算正規化因子並應用
        normalized_audio = audio / max_amplitude * target_peak
        
        # 使用 soundfile 覆蓋原始檔案，保留原始取樣率
        sf.write(audio_path, normalized_audio, sr)
        print(f"    - ✅ 音量已成功正規化!")

    except Exception as e:
        print(f"    - ❌ 正規化失敗: {os.path.basename(audio_path)}，錯誤: {e}")


def convert_and_normalize_all_subdirs(base_dir, sound_font_path):
    """
    遞迴掃描指定的基本目錄，找到所有名為 'piano.mid' 的檔案，
    將其轉換為 'picogen-e.wav'，並立即進行音量正規化。

    Args:
        base_dir (str): 要掃描的根目錄，應包含 CPOP, JPOP 等子資料夾。
        sound_font_path (str): SoundFont (.sf2) 檔案的路徑。
    """
    # --- 1. 檢查路徑是否存在 ---
    if not os.path.isdir(base_dir):
        print(f"❌ 錯誤：指定的根目錄不存在 -> {base_dir}")
        return
    if not os.path.exists(sound_font_path):
        print(f"❌ 錯誤：SoundFont 檔案不存在 -> {sound_font_path}")
        return

    print(f"🚀 開始掃描目錄: {base_dir}")
    print(f"🎵 使用音色庫: {sound_font_path}")

    # --- 2. 遍歷所有子目錄 ---
    total_processed = 0
    total_skipped = 0
    
    for dirpath, _, filenames in os.walk(base_dir):
        if 'piano.mid' in filenames:
            input_midi_path = os.path.join(dirpath, 'piano.mid')
            output_wav_path = os.path.join(dirpath, 'picogen-e.wav')

            # if os.path.exists(output_wav_path):
            #     total_skipped += 1
            #     continue

            try:
                print(f"\n🔄 正在處理: {input_midi_path}")
                
                # 步驟 A: 執行 MIDI -> WAV 轉換
                print(f"  🎶 轉換為 -> {os.path.basename(output_wav_path)}")
                midi_to_wav(input_midi_path, output_wav_path, sound_font_path)
                
                # 步驟 B: 立刻對剛生成的 WAV 檔案進行音量正規化
                normalize_wav_file(output_wav_path)
                
                total_processed += 1
            except Exception as e:
                print(f"  ❌ 處理失敗: {input_midi_path}，錯誤: {e}")

    print("\n🎉 --- 所有任務已完成！ ---")
    print(f"總共處理 (轉換並正規化) 了 {total_processed} 個新檔案。")
    print(f"總共跳過了 {total_skipped} 個已存在的檔案。")


# --- 主程式執行區塊 ---
if __name__ == '__main__':
    # --- 請在此處設定您的路徑 ---
    
    # 1. 設定包含 CPOP, JPOP, KPOP, Western 的最上層目錄
    #    例如，如果您的結構是 ./data/CPOP/...，這裡就填 ./data
    root_dataset_dir = os.path.join(".", "user") # <--- 請修改為您的主要資料夾

    # 2. 設定 SoundFont (.sf2) 檔案的路徑
    sound_font_file = os.path.join(".", "utils", "sound_font", "SGM-v2.01.sf2")

    # --- 執行轉換 ---
    convert_and_normalize_all_subdirs(base_dir=root_dataset_dir, sound_font_path=sound_font_file)

In [ ]:
import json
import os
from corpus import Synchronizer

def batch_process_warping_paths():
    """
    讀取 metadata.json，對每首歌的四個版本與 origin.wav 計算對齊路徑(wp)，
    並利用 Synchronizer 的快取機制自動儲存結果到各目錄的 wp.json 中。
    """
    # --- 1. 設定路徑和常數 ---
    base_dir = os.path.join(".", "dataset", "eval")
    metadata_path = os.path.join(base_dir, "metadata.json")
    origin_filename = "origin.wav"
    
    # 定義要處理的四個翻奏版本名稱
    cover_versions = ["human", "picogen", "amtapc", "music2midi", "etude_e", "etude_d", "etude_d_d"]

    # --- 2. 讀取 metadata.json ---
    try:
        with open(metadata_path, 'r', encoding='utf-8') as f:
            metadata = json.load(f)
        print(f"✅ 成功讀取 metadata.json，共找到 {len(metadata)} 首歌曲。")
    except FileNotFoundError:
        print(f"❌ 錯誤：找不到 metadata.json 檔案，請確認路徑 '{metadata_path}' 是否正確。")
        return

    # --- 3. 實例化 Synchronizer ---
    # 可以在迴圈外實例化一次，重複使用
    synchronizer = Synchronizer()

    # --- 4. 遍歷每首歌曲和每個版本進行計算 ---
    for i, song_data in enumerate(metadata):
        dir_name = song_data.get("dir_name")
        if not dir_name:
            print(f"⚠️ 警告：第 {i+1} 筆資料缺少 'dir_name'，已跳過。")
            continue

        song_dir = os.path.join(base_dir, dir_name)
        print(f"\n🎵 === 處理歌曲目錄: {song_dir} ({i+1}/{len(metadata)}) ===")

        # 檢查 origin.wav 是否存在
        origin_wav_path = os.path.join(song_dir, origin_filename)
        if not os.path.exists(origin_wav_path):
            print(f"  ❌ 錯誤：找不到基準檔案 {origin_wav_path}，無法進行比較，已跳過此目錄。")
            continue

        # 遍歷需要處理的四個版本
        for version_name in cover_versions:
            print(f"  --- 版本: {version_name} ---")
            cover_wav_path = os.path.join(song_dir, f"{version_name}.wav")

            # 檢查 cover audio 是否存在
            if not os.path.exists(cover_wav_path):
                print(f"  ↪️ 已跳過 (WAV 不存在): {cover_wav_path}")
                continue
            
            try:
                synchronizer.get_wp(origin_wav_path, cover_wav_path, song_dir)
            except Exception as e:
                print(f"  ❌ 處理失敗: {cover_wav_path}，錯誤: {e}")

    print("\n🎉🎉🎉 --- 所有歌曲目錄的對齊路徑(wp)都已處理完成！ ---")


# --- 執行主程式 ---
if __name__ == '__main__':
    batch_process_warping_paths()

https://youtu.be/6Q0Pd53mojY\?si\=dJuemwnwfdlsgbfk
https://youtu.be/Ug5-kXqP5l8\?si\=WwW9_D6QyBSO6cXZ
https://youtu.be/iFIXi6zzCls?si=kYxheOnqR3573IZp
https://youtu.be/kbNdx0yqbZE?si=4Ze8lkq-LGflsvJE
https://youtu.be/OLRbIc8KZ_8?si=nCbfnyRqRdofOudC 
https://youtu.be/s1bZEnGAX8I\?si\=LigiA3P9sxbBNFwj (musicxml error)
https://youtu.be/4MoRLTAJY_0\?si\=QvLpDCztTiz_wWIT (beat detection error)
https://youtu.be/wgwIfD9Ihik?si=Pap3maz0ho4fv16v
https://youtu.be/JQ2913bVo30?si=7TNGouF9baF_iZWg
https://youtu.be/zjEMFuj23B4?si=kmmHvJ4Wh-ariHIn

#### CPOP

- https://youtu.be/OLRbIc8KZ_8?si=HTPDSGHKPtESid2G 2 3 4
- https://youtu.be/in8NNzwFa-s?si=A9BuyurRE4UPfNtJ
- https://youtu.be/HQ_mU73VhEQ?si=z0Qgj89QVm0P6bVl 4 4 4
- https://youtu.be/8MG--WuNW1Y\?si\=6Y38ZiA2l7ZdKtd9 3 4 4
- https://youtu.be/ZPALMaXLfIw?si=MCc7w7vwrDdlDhNm 3 3 3
- https://youtu.be/h0qYPIlE9us?si=mTG3oz-52Ou2bOi7 2 3 2

---
#### JPOP
https://youtu.be/kbNdx0yqbZE?si=B40NV4X87AqOyx5g 2 2 2
https://youtu.be/Yq7e_AY0dnk?si=KZi6YMpaT6CIWs_g 2 3 4
https://youtu.be/M-Eyhjkepy0?si=wztq11Kp4xCHOqjz 2 3 3
https://youtu.be/fp3F6TqBsAU\?si\=QJ_QvqZoH4HXvf2R 2 2 3
https://youtu.be/bVUEuXOjeDc?si=Wq-ujMPc8qWHnwi3
https://youtu.be/XwgL4C2WaU8?si=9yP_3iEI4YGhfUdr

In [ ]:
import os
import shutil

# --- 設定 ---
# 來源基礎目錄
source_base_dir = './dataset/eval'
# 目標基礎目錄
dest_base_dir = './songs'
# 要複製的檔案名稱
filename_to_copy = 'wp.json'
# --- 設定結束 ---

# 檢查來源和目標基礎目錄是否存在
if not os.path.isdir(source_base_dir):
    print(f"錯誤：來源目錄 '{source_base_dir}' 不存在。請檢查路徑是否正確。")
    exit()

if not os.path.isdir(dest_base_dir):
    print(f"錯誤：目標目錄 '{dest_base_dir}' 不存在。請檢查路徑是否正確。")
    exit()

print(f"開始從 '{source_base_dir}' 複製 '{filename_to_copy}' 到 '{dest_base_dir}'...")

# 獲取來源目錄下的所有項目名稱（檔案和目錄）
try:
    subdirs = os.listdir(source_base_dir)
except FileNotFoundError:
    print(f"錯誤：無法讀取來源目錄 '{source_base_dir}'。")
    exit()

# 初始化計數器
copied_count = 0
skipped_count = 0
error_count = 0

# 遍歷所有在來源目錄下的子目錄
for subdir_name in subdirs:
    source_subdir_path = os.path.join(source_base_dir, subdir_name)
    
    # 確保我們只處理目錄
    if not os.path.isdir(source_subdir_path):
        continue

    # 構造完整的來源檔案路徑和目標目錄路徑
    source_file_path = os.path.join(source_subdir_path, filename_to_copy)
    dest_dir_path = os.path.join(dest_base_dir, subdir_name)

    # 1. 檢查來源檔案是否存在
    if not os.path.exists(source_file_path):
        print(f"資訊：跳過 '{subdir_name}'，因為找不到來源檔案 '{filename_to_copy}'。")
        skipped_count += 1
        continue

    # 2. 檢查目標目錄是否存在
    if not os.path.exists(dest_dir_path):
        print(f"資訊：跳過 '{subdir_name}'，因為目標目錄 '{dest_dir_path}' 不存在。")
        skipped_count += 1
        continue

    # 3. 執行複製操作
    try:
        shutil.copy2(source_file_path, dest_dir_path)
        print(f"成功：已將 '{subdir_name}/{filename_to_copy}' 複製到 '{dest_dir_path}'")
        copied_count += 1
    except Exception as e:
        print(f"錯誤：複製檔案到 '{dest_dir_path}' 時發生錯誤: {e}")
        error_count += 1

print("\n--- 任務完成 ---")
print(f"成功複製 {copied_count} 個檔案。")
print(f"因來源檔案或目標目錄不存在而跳過 {skipped_count} 個。")
print(f"複製過程中發生 {error_count} 次錯誤。")